# IFC Rasterization for Efficient Spatial Analysis

## Intro

In [4]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
from concurrent.futures import ThreadPoolExecutor


def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:

    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3

    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3) 
    return faces

def vtk_block_by_building_element(ifc_file):
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    exclude_list = ["IfcSpace", "IfcOpeningElement"]
    
    multiblock = pv.MultiBlock()

    if iterator.initialize():

        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)

            if not iterator.next():
                break
               
    return multiblock

class OctreeNode:
    def __init__(self, bounds, level=0):
        self.bounds = bounds
        self.level = level
        self.children = []
        self.is_leaf = True
        self.contains_mesh = False

class Octree:
    def __init__(self, bounds, max_levels):
        self.root = OctreeNode(bounds, level=0)
        self.max_levels = max_levels

def object_fully_contained_in_node(node, mesh):
    node_bounds = node.bounds
    mesh_bounds = mesh.bounds
    return all(node_bounds[i] <= mesh_bounds[i] <= node_bounds[i+1] for i in range(0, 6, 2))

def object_intersects_node(node, mesh):
    node_bounds = node.bounds
    mesh_bounds = mesh.bounds
    return all(node_bounds[i] <= mesh_bounds[i+1] and node_bounds[i+1] >= mesh_bounds[i] for i in range(0, 6, 2))

def subdivide_node(node):
    bounds = node.bounds
    level = node.level
    x_mid = (bounds[0] + bounds[1]) / 2
    y_mid = (bounds[2] + bounds[3]) / 2
    z_mid = (bounds[4] + bounds[5]) / 2

    new_bounds = [
        [bounds[0], x_mid, bounds[2], y_mid, bounds[4], z_mid],
        [x_mid, bounds[1], bounds[2], y_mid, bounds[4], z_mid],
        [bounds[0], x_mid, y_mid, bounds[3], bounds[4], z_mid],
        [x_mid, bounds[1], y_mid, bounds[3], bounds[4], z_mid],
        [bounds[0], x_mid, bounds[2], y_mid, z_mid, bounds[5]],
        [x_mid, bounds[1], bounds[2], y_mid, z_mid, bounds[5]],
        [bounds[0], x_mid, y_mid, bounds[3], z_mid, bounds[5]],
        [x_mid, bounds[1], y_mid, bounds[3], z_mid, bounds[5]],
    ]

    node.children = [OctreeNode(b, level=level+1) for b in new_bounds]
    node.is_leaf = False

def insert_mesh(node, mesh, max_levels):
    if node.level == max_levels or object_fully_contained_in_node(node, mesh):
        node.contains_mesh = True
    else:
        if node.is_leaf:
            subdivide_node(node)
        for child in node.children:
            if object_intersects_node(child, mesh):
                insert_mesh(child, mesh, max_levels)

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))


def voxelize_space(meshes, voxel_size, bounds):
    
    """Voxelize space and check intersections with given mesh."""

    grid = create_uniform_grid(bounds, voxel_size)

    num_points = grid.cell_centers().n_points
    mask = np.zeros(num_points, dtype=bool)

    print(f'Total number of voxels: {num_points}')

    select_enclosed = vtk.vtkSelectEnclosedPoints()
    select_enclosed.SetInputData(grid.cell_centers())

    for mesh in meshes:
        select_enclosed.SetSurfaceData(mesh)
        select_enclosed.Update()
        mask = np.logical_or(mask, np.array([select_enclosed.IsInside(i) for i in range(num_points)]))

    neighbour_list = []
    filled_indices = []
    
    # get the indices of the filled cells and their neighbours to minimize the search space
    for i in range(num_points):
        if mask[i]==1:
            neighbour_list.extend(grid.cell_neighbors(i, "edges"))
            filled_indices.append(i)

    neighbour_list = list(set(neighbour_list)) #remove duplicates
    print(len(neighbour_list))
    print(len(filled_indices))

    for ind in neighbour_list: #only checking the neighbours of the filled cells
        if mask[ind] == 0: #if the cell is not filled
            #look at the surrounding cells and check if they are filled
            neighbours = grid.cell_neighbors(ind, "edges") 

            print(f'cell_index = {ind}')
            print(f'neighbour_indices = {neighbours}')
            
            neighbour_x = []
            neighbour_y = []
            neighbour_z = []

            for n in neighbours:   #check if the neighbours are in the same plane as the cell, if they are filled
                if mask[n] == 1:
                    if grid.cell_centers().points[ind][0] == grid.cell_centers().points[ind][0]:
                        neighbour_x.append(n)
                    elif grid.cell_centers().points[ind][1] == grid.cell_centers().points[ind][1]:
                        neighbour_y.append(n)
                    elif grid.cell_centers().points[ind][2] == grid.cell_centers().points[ind][2]:
                        neighbour_z.append(n)
                
            if len(neighbour_x) == 2: 
                mask[ind] = 1   
                
            elif len(neighbour_y) == 2: 
                mask[ind] = 1 
                
            elif len(neighbour_z) == 2:
                mask[ind] = 1
                

    return grid, mask

def voxelize(node, voxel_size, voxelized_representation, meshes):
    if node.is_leaf:
        if node.contains_mesh:
            # Perform voxelization in this region using voxelize_space function
            grid, mask = voxelize_space(meshes, voxel_size, node.bounds)
            voxelized_representation.append((grid, mask))
    else:
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(voxelize, child, voxel_size, voxelized_representation, meshes) for child in node.children]
            for future in futures:
                future.result()


## Importing IFC objects

In [5]:
ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Clinic.ifc")

# convert IFC objects into vtk meshes
start_time = time.time()

all_meshes = vtk_block_by_building_element(ifc_file)

end_time = time.time()
print(f"Time taken to convert objects: {end_time - start_time:.4f} seconds")

Time taken to convert objects: 0.1220 seconds


## Voxelization

In [6]:
start_time = time.time()

# Create the octree
bounds = all_meshes.bounds # You'll need to determine these based on your data
max_levels = 6 # Adjust based on your needs
octree = Octree(bounds, max_levels)

#merged_polydata = reduce(lambda x, y: x + y, all_meshes)
for mesh in all_meshes:
    insert_mesh(octree.root, mesh, octree.max_levels)

# Perform the voxelization
voxel_size = 0.2  # Adjust based on your needs
voxelized_representation = []
voxelize(octree.root, voxel_size, voxelized_representation, all_meshes)

end_time = time.time()
print(f"Time taken to voxelize: {end_time - start_time:.4f} seconds")

# Visualization
p = pv.Plotter()
    
for grid, mask in voxelized_representation:
    #mask = update_cell_values(grid)
    p.add_mesh(grid.extract_cells(np.where(mask)[0]), color="blue", show_edges=True, opacity=0.5)
    
p.add_mesh(grid,show_edges=True, opacity=0.3)

p.show()


Total number of voxels: 10944
3552
1821
cell_index = 23
neighbour_indices = [46, 623, 47, 598, 599, 22]
cell_index = 47
neighbour_indices = [70, 71, 647, 46, 623, 622, 23, 22, 599]
cell_index = 71
neighbour_indices = [70, 647, 646, 46, 623, 47, 671, 94, 95]
cell_index = 95
neighbour_indices = [70, 647, 71, 695, 670, 118, 119, 94, 671]
cell_index = 119
neighbour_indices = [142, 143, 719, 118, 695, 694, 94, 95, 671]
cell_index = 143
neighbour_indices = [166, 167, 743, 142, 719, 718, 119, 118, 695]
cell_index = 167
neighbour_indices = [166, 743, 742, 142, 719, 143, 767, 190, 191]
cell_index = 191
neighbour_indices = [166, 743, 167, 791, 766, 214, 215, 190, 767]
cell_index = 215
neighbour_indices = [191, 238, 239, 815, 214, 791, 790, 190, 767]
cell_index = 239
neighbour_indices = [262, 263, 839, 238, 815, 814, 214, 791, 215]
cell_index = 263
neighbour_indices = [262, 839, 838, 238, 815, 239, 863, 286, 287]
cell_index = 287
neighbour_indices = [262, 839, 263, 887, 862, 310, 311, 286, 863]
c

Widget(value="<iframe src='http://localhost:64624/index.html?ui=P_0x1f74475d540_0&reconnect=auto' style='width…